## Rag (Method 1)

### Loading Doc

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('NL_January2024.pdf')
docs=loader.load()

#### Text preprocessing function

In [2]:
def store_string_to_text_file(string_to_store, file_path):

  with open(file_path, 'w') as file:
    file.write(string_to_store)


file_name = "answer.txt"

### Text Splitting

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)

In [4]:
chunks = text_splitter.split_documents(docs)

### Embedding And Storing

In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embedding_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

c:\Users\Arjun\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Arjun\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# vector_db = Chroma.from_documents(documents=chunks,
#                                   embedding=embedding_model,
#                                   persist_directory="data_base_1",
#                                   )


##### Load VectorDB (if have)

In [8]:
vector_db = Chroma(persist_directory="data_base_1",
                   embedding_function=embedding_model)

c:\Users\Arjun\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(
c:\Users\Arjun\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [9]:
retriver = vector_db.as_retriever(search_kwargs={'k': 3})

### Testing Retriever

In [10]:
vector_db.similarity_search("where run for unity organized ?", k=3)

[Document(metadata={'page': 8, 'source': 'NL_January2024.pdf'}, page_content="9\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nRUN FOR UNITY\nDL, Jodhpur\nOn the Rashtriya Ekta Diwas \norganised on 31 October 2023, \nall\tthe\tofficers\t and\tstaff\tof\tthe\t\nDefence Laboratory, Jodhpur (DLJ) took the Rashtriya Ekta Diwas Pledge. Further, ‘Run for Unity’ was organised to spread the message of unity in society. The event was led by Shri RV Hara \nPrasad, Outstanding Scientist & \nDirector, DLJ. During his address, he highlighted the contributions made by the legend Sardar Vallabh Bhai Patel in the freedom struggle \nand national integration after \nindependence.\nISSA, Delhi \nOn 31 October 2023, Institute \nfor Systems Studies & Analyses \n(ISSA), Delhi, organised ‘Run for Unity’ from Metcalfe House to Qudasia Bagh, near ISBT \nKashmere Gate. It is an 18th-\ncentury garden complex with depictions of the Indian freedom struggle movement. \nAll\tofficers\t and\tstaff\tof\tthe\t\nlaboratory t

### Prompt Passing

In [11]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")


In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """ 
    Answer the following question based on the provided context.
    Note : analyse given context carefully
    <context>
    {context}
    </context>

    Question : {input}""")


In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
from langchain.chains import create_retrieval_chain
retrival_chain = create_retrieval_chain(retriver, document_chain)

##### Question 1

In [26]:
ans = retrival_chain.invoke({"input" : "List of all the places where RUN FOR UNITY was organized"})

In [27]:
ans

{'input': 'List of all the places where RUN FOR UNITY was organized',
 'context': [Document(metadata={'page': 8, 'source': 'NL_January2024.pdf'}, page_content="9\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nRUN FOR UNITY\nDL, Jodhpur\nOn the Rashtriya Ekta Diwas \norganised on 31 October 2023, \nall\tthe\tofficers\t and\tstaff\tof\tthe\t\nDefence Laboratory, Jodhpur (DLJ) took the Rashtriya Ekta Diwas Pledge. Further, ‘Run for Unity’ was organised to spread the message of unity in society. The event was led by Shri RV Hara \nPrasad, Outstanding Scientist & \nDirector, DLJ. During his address, he highlighted the contributions made by the legend Sardar Vallabh Bhai Patel in the freedom struggle \nand national integration after \nindependence.\nISSA, Delhi \nOn 31 October 2023, Institute \nfor Systems Studies & Analyses \n(ISSA), Delhi, organised ‘Run for Unity’ from Metcalfe House to Qudasia Bagh, near ISBT \nKashmere Gate. It is an 18th-\ncentury garden complex with depictions of the India

In [28]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

'Based on the provided context, "Run for Unity" was organized at the following places:    1. Defence Laboratory (DL), Jodhpur  2. Institute for Systems Studies & Analyses (ISSA), Delhi  3. Microwave Tube Research & Development Centre (MTRDC), Bengaluru  4. Naval Science & Technological Laboratory (NSTL), Visakhapatnam'

In [24]:

store_string_to_text_file(ans['answer'], file_name)

##### Question 2

In [20]:
ans2 = retrival_chain.invoke({"input" : "List of all the places where Vigilance Awareness Week was organized"})

In [21]:
ans2

{'input': 'List of all the places where Vigilance Awareness Week was organized',
 'context': [Document(metadata={'page': 7, 'source': 'NL_January2024.pdf'}, page_content="8\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nVIGILANCE AWARENESS WEEK\nVigilance Awareness Week is observed every year during the week in which the birthday of \nSardar Vallabhbhai Patel (31 October) falls. The theme of this year’s Vigilance Awareness Week was ‘Say No to Corruption, Commit to the Nation’.\nDL, Jodhpur\nVigilance Awareness Week was \ncelebrated during 30 October to 3 November 2023 at the Defence \nLaboratory, Jodhpur (DLJ). All \nthe\tofficers\tand\tstaff\ttook\ta\tpledge\t\nregarding vigilance. A lecture on 'Say No to Corruption: Commit \nto the Nation' was delivered by \nShri Harendra Kumar Mahawar, \nIPS, IG, Anti-Corruption Bureau, \nJodhpur, on the occasion.\nIRDE, Dehradun\nInstruments Research & \nDevelopment Establishment (IRDE), Dehradun, observed \nVigilance Awareness Week during \n30 October 

In [19]:
st2 = ans2['answer']
stt2 = st2.replace("\n", "  ")
stt2

'Based on the provided context, the list of places where Vigilance Awareness Week was organized are:    1. Defence Laboratory (DL), Jodhpur  2. Instruments Research & Development Establishment (IRDE), Dehradun'

##### Question 3

In [20]:
ans = retrival_chain.invoke({"input" : "Give me the summary of KARNATAKA RAJYOTSAVA CELEBRATION"})

In [21]:
ans

{'input': 'Give me the summary of KARNATAKA RAJYOTSAVA CELEBRATION',
 'context': [Document(metadata={'page': 10, 'source': 'NL_January2024.pdf'}, page_content='11\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\n(NPOL), Kochi celebrated its \n71st Annual Day on 15 December 2023. Dr S Unnikrishnan Nair, DS \n& Director, VSSC was the Chief \nGuest for the event. Dr R Rajesh, Sc ‘G’ & Chairman, NPOL Annual Day Organising Committee welcomed the gathering and presented an overview of the \nfunction. Dr Ajith Kumar K, OS \n& Director, NPOL presented the report on the achievements of the laboratory in both technical and non-technical areas. Shri \nWilson K Cherukulath, Sc ‘G’ & \nChairman, Works Committee & Secretary, Works Committee and General Secretaries of NPOL’s Civilian Employees union and NPOL’s R&D Employees union \nalso spoke on the occasion. Prizes \nwere distributed for the sports competitions held during the Annual Day. Instrumental music fusion by ‘RLV College of Music \nand Fine Arts’ 

In [22]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

'Here is a summary of the KARNATAKA RAJYOTSAVA CELEBRATION:    The Microwave Tube Research & Development Centre (MTRDC) in Bengaluru celebrated the Karnataka State Formation Day, also known as Rajyotsava, on November 17, 2023. The event was attended by Dr. Ganesha Bhatt, a retired principal, who spoke about the rich heritage and classical language of Kannada. Other attendees included the Centre Head of MTRDC, Director of LRDE & MTRDC, and staff from MTRDC. A cultural program was also organized, featuring a flute performance by Shri Srinidhi Katti, a professional flautist. The event concluded with a vote of thanks proposed by Shri Jayateertha D, an engineer at MTRDC.'

##### Question 4

In [15]:
ans = retrival_chain.invoke({"input" : "List of all the places where SWACHHATA PAKHWADA was organized"})

In [16]:
ans

{'input': 'List of all the places where SWACHHATA PAKHWADA was organized',
 'context': [Document(metadata={'page': 13, 'source': 'NL_January2024.pdf'}, page_content='14\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nSWACHHATA PAKHWADAwas invited as a guest speaker on \n05 Decemebr 2023. She delivered a lecture on ‘Importance of Millets’ \nand\tfocused\t on\tthe\tbenefits\t of\t\nmillets in the diet. Shri PT Rojatkar, \nOS & Director, NMRL felicitated the speaker. Cooking and poster competitions on millets were also organised on 11 December \n2023. NMRL employees and their \nfamily members took part in the competition. Over 200 people visited the exhibition where the posters were displayed. The \nwinners were awarded with the \nprices\tand\tcertificates.\nDESIDOC, Delhi\nThe\t Defence\t Scientific\t\nInformation & Documentation Centre (DESIDOC), Delhi, \norganised a swachhata Pakhwada \nduring 01-15 December 2023. Besides the cleanliness drive during the two-week-long program, a couple of ev

In [17]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

'Based on the provided context, the list of places where SWACHHATA PAKHWADA was organized is:    1. DESIDOC, Delhi  2. Defence Laboratory, Jodhpur (DLJ)  3. EMU, Hyderabad  4. Instruments Research & Development Establishment (IRDE), Dehradun  5. SF Complex (SFC), Jagdalpur'

##### Question 5

In [24]:
ans = retrival_chain.invoke({"input" : "Give me brief of International Year of Millets"})

In [25]:
ans

{'input': 'Give me brief of International Year of Millets',
 'context': [Document(metadata={'page': 12, 'source': 'NL_January2024.pdf'}, page_content="13\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nDRDO PARTICIPATION IN BIHAR SCIENTIFIC \nLITERACY FESTIVAL\nINTERNATIONAL YEAR OF MILLETS-2023The Defence Research & \nDevelopment Organisation \n(DRDO) participated in the \nBihar\tScientific\t Literacy\t Cum\t\nHealth & Wellness Festival held \nat Patna, Biharunder Patliputra Lok Sabha constituency, during 07-09 December 2023. The event was supported by DAE, ICMR, \nISRO, CSIR, MOES, DST, and \nDRDO. The three-day educational and information dissemination outreach and sensitization event was inaugurated by Shri SK Jha, \nZonal Director Education, Patna \nZone-A, in the august presence \n \nDr Krishna Pandey, Sc ‘G’ & Director, Rajendra Memorial Research Institute of Medical \nSciences, Patna, and other \ndignitaries representing the district and state administration and other \nITM, Mussoori

In [26]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

"Based on the provided context, here's a brief summary of the International Year of Millets:    * The International Year of Millets (IYM) was celebrated in 2023.  * The IYM aimed to raise awareness about the benefits of millets, which are small-grained cereals that are nutritious and environmentally friendly.  * The celebration involved various events and activities across different organizations, including the Defence Research & Development Organisation (DRDO), Naval Materials Research Laboratory (NMRL), Institute of Technology Management (ITM), and others.  * Events included lectures, quizzes, competitions, and exhibitions showcasing the nutritional, economic, and ecological benefits of millets.  * The IYM emphasized the importance of incorporating millets into daily diets for a healthier and more sustainable lifestyle."

##### Question 6

In [32]:
ans = retrival_chain.invoke({"input" : "who is from Aligarh Muslim University shri Parimal Kumar"})

In [33]:
ans

{'input': 'who is from Aligarh Muslim University shri Parimal  Kumar',
 'context': [Document(metadata={'page': 13, 'source': 'NL_January2024.pdf'}, page_content='14\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nSWACHHATA PAKHWADAwas invited as a guest speaker on \n05 Decemebr 2023. She delivered a lecture on ‘Importance of Millets’ \nand\tfocused\t on\tthe\tbenefits\t of\t\nmillets in the diet. Shri PT Rojatkar, \nOS & Director, NMRL felicitated the speaker. Cooking and poster competitions on millets were also organised on 11 December \n2023. NMRL employees and their \nfamily members took part in the competition. Over 200 people visited the exhibition where the posters were displayed. The \nwinners were awarded with the \nprices\tand\tcertificates.\nDESIDOC, Delhi\nThe\t Defence\t Scientific\t\nInformation & Documentation Centre (DESIDOC), Delhi, \norganised a swachhata Pakhwada \nduring 01-15 December 2023. Besides the cleanliness drive during the two-week-long program, a couple of events

In [34]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

"There is no information provided about Shri Parimal Kumar being from Aligarh Muslim University. In fact, there is no mention of Shri Parimal Kumar at all in the given context. Therefore, it's not possible to answer this question based on the provided context. If you meant to ask a different question or provide more context, I'd be happy to help!"

##### Question 7

In [35]:
ans = retrival_chain.invoke({"input" : "what happen in DRDO NORTH ZONE KABADDI TOURNAMENT ?"})

In [36]:
ans

{'input': 'what happen in DRDO NORTH ZONE KABADDI TOURNAMENT ?',
 'context': [Document(metadata={'page': 21, 'source': 'NL_January2024.pdf'}, page_content='22\n|   JANUARY 2024\nDRDO NEWSLETTERSPORTS NEWS\nDRDO NORTH ZONE CARROM TOURNAMENT\nThe Defence Geoinformatics \nResearch Establishment (DGRE), \nChandigarh, hosted the DRDO \nNorth Zone Carrom Tournament \nduring 28 November 2023 to 01 December 2023. Dr PK Satyawali, OS & Director, DGRE, inaugurated the event and urged all the players with zeal and enthusiasm.  \nShri Niranjan Kumar, DESIDOC, won the Men’s Singles; Shri Vikas Kharola and Surendra Singh, IRDE, won the Men’s Doubles; Ms Supriya Aherwar, JCB, won \nthe Women’s Singles; Ms Monica \nTondon and Ms Usha Kumari, IRDE, won the Women’s Doubles; Shri NK Balodi, DEAL, won the Veterans Single; and Shri Bharat Singh and Shri Bhagat Ram, IRDE, \nwon the Veterans Doubles. \nDRDO NORTH ZONE KABADDI TOURNAMENT \nThe Defence Materials and \nStores Research & Development \nEstablishm

In [37]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

"Based on the provided context, here's the answer:    The DRDO North Zone Kabaddi Tournament 2023 was organized by the Defence Materials and Stores Research & Development Establishment (DMSRDE), Kanpur, during 06-09 November 2023. The tournament saw participation from seven teams from sister laboratories, namely TBRL, INMAS, CFEES, DEAL, IRDE, SAG, and host DMSRDE.    The winner of the tournament was TBRL, Chandigarh, who defeated INMAS, Delhi, in the final. Three players from TBRL, two players from INMAS, and one player each from DEAL, IRDE, SAG, CFEES, and DMSRDE were selected to represent the North Zone during the DRDO Nationals tournament.    The Chief Guest for the inaugural event was Dr Mayank Dwivedi, OS & Director, DMSRDE. The tournament was organized by Dr Ashish Dubey, Sc 'F' & Sports Officer."

: 